<a href="https://colab.research.google.com/github/Anjana2002/CodeMix-Language-Detector/blob/main/data%20annotation/annotation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## DATA ANNOTATION

In [ ]:
!pip install pandas nltk

In [ ]:
!pip install spacy

In [ ]:
!pip install langdetect

In [ ]:
import requests
import pandas as pd
import re
import nltk
import spacy

from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import words, stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('words')
nltk.download('stopwords')
nltk.download('punkt_tab')

df = pd.read_csv("malayalam_youtube_comments_filtered.csv")
df['text'] = df['text'].str.lower()
df.head()

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


,video_id,text
0,63i4ZQcvpcQ,nayanthara kettiyathin shesham kanunnavar
1,63i4ZQcvpcQ,00:12 which is that song playing in the backgr...
2,63i4ZQcvpcQ,2024 july 21n kaanunna njan🥲🥲
3,63i4ZQcvpcQ,15:57 what she is saying
4,63i4ZQcvpcQ,😊 ara e script oke ezhuthiye..nice dialogues a...


In [ ]:
url = "https://inventwithpython.com/dictionary.txt"
response = requests.get(url)
if response.status_code == 200:
    external_eng_words = set(word.lower() for word in response.text.splitlines())
    print(f"Loaded {len(external_eng_words)} words from the external dictionary.")
else:
    print("Failed to fetch the dictionary. Check the URL.")
    external_eng_words = set()

Loaded 45304 words from the external dictionary.


In [ ]:
genz_words_df = pd.read_csv("genz_slang.csv")
genz_words_df = genz_words_df[["keyword"]]
genz_words = set(genz_words_df["keyword"].str.lower())

In [ ]:

stop_words_set = set(stopwords.words('english'))  # Stopwords in English
eng_word_set = external_eng_words | stop_words_set | genz_words
emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # Emoticons
                           u"\U0001F300-\U0001F5FF"  # Symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # Transport & map symbols
                           u"\U0001F700-\U0001F77F"  # Alchemical symbols
                           u"\U0001F780-\U0001F7FF"  # Geometric shapes extended
                           u"\U0001F800-\U0001F8FF"  # Supplemental arrows-C
                           u"\U0001F900-\U0001F9FF"  # Supplemental symbols and pictographs
                           u"\U0001FA00-\U0001FA6F"  # Chess symbols
                           u"\U0001FA70-\U0001FAFF"  # Symbols and pictographs extended-A
                           u"\U00002702-\U000027B0"  # Dingbats
                           u"\U000024C2-\U0001F251"  # Enclosed characters
                           u"\U0001F1E0-\U0001F1FF"  # Flags
                           "]+", flags=re.UNICODE)

In [ ]:
acronyms_df = pd.read_csv("acronyms.csv")
acronyms_df['acronyms'] = acronyms_df['acronyms'].str.strip().str.lower()  # Normalize to lowercase
acronyms = set(acronyms_df['acronyms'])
universal_symbols = {
    "#", "@", "&", "%", "*", "!", "$", "^", "(", ")", "-", "_", "=", "+",
    "[", "]", "{", "}", "|", "\\", ":", ";", "'", '"', "<", ">", ",", ".",
    "?", "/", "~", "`"
}

malayalam_suffixes = [
    'il', 'ilude', 'in', 'inu', 'inte', 'nte', 'kk', 'kkum', 'ilum', 'kalil', 'u'
    'inte', 'nte', 'kku', 'kkoru', 'kkulla', 'te', 'muttam',
    'kal', 'kale', 'kkal', 'kkale', 'kkarum', 'kkalum', 'kalude', 'kkarum',
    'um', 'avum', 'athum', 'ennum', 'athavum', 'inum', 'anum', 'aayum',
    'unnu', 'an', 'iyum', 'aayi', 'aayirunnu', 'aarunnu', 'ppar', 'kunnu', 'ittundu',
    'umayi', 'ittu', 'ettum', 'aarilla', 'illa', 'athilla',
    'aayi', 'ayi', 'aayitt', 'pol', 'ppol', 'umayi', 'aakkum', 'ollu', 'kaaran',
    'o', 'aano', 'alle', 'athano', 'aano', 'ente', 'ennu', 'athinte',
    'ill', 'illa', 'athill', 'arilla', 'athumill',
    'onnu', 'nnulla', 'ullathu', 'ullavan', 'ullavarkku', 'ullathu', 'athinu',
    'tharam', 'padi', 'pozhuthu', 'athre', 'polathe', 'kku',
    'ennu', 'aayi', 'aakunnu', 'ennu', 'arut', 'aaru', 'ppoloru', 'athin',
    'aakathirikkan', 'thannullathu', 'poyi', 'irikkum', 'vannathum',
    'ppol', 'kalath', 'kaalath', 'nirath', 'kaalam', 'samayam', 'pozhuthu', 'pozhuthil', 'al',
    'ayi', 'aayi', 'aayirunnu', 'aayitt',
    'um', 'u', 'unnu', 'aayi', 'avum', 'alle', 'an', 'onnu', 'ill','yil','aakk', 'ippol', 'onnum', 'eth', 'am', 'kku', 'ppol', 'umayi', 'kaaran'
]

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
def word_length(word):
    return len(word)

def is_mixed(word):
    """Checks if the word contains a mix of letters and numbers."""
    return any(c.isalpha() for c in word) and any(c.isdigit() for c in word)

def is_mixed_word_language_model(word, english_words, malayalam_suffixes):
    word = word.lower()

    # Start with the full word and reduce it iteratively
    for i in range(len(word), 0, -1):
        prefix = word[:i]
        suffix = word[i:]


        if prefix in english_words:
            if suffix in malayalam_suffixes:
                return True

    return False


In [ ]:
def tag_comment(comment):
    """Tags a single comment at the word level."""
    temp_comments = []
    words = word_tokenize(comment)  # Tokenize the comment

    for word in words:
        word = word.lower()


        if word.isnumeric():  # Check if the word is a number
            temp_comments.append((word, "univ"))

        elif word in eng_word_set:
            temp_comments.append((word, "eng"))

        elif emoji_pattern.match(word):
            temp_comments.append((word, "univ"))

        elif word in acronyms:
            temp_comments.append((word, "acr"))

        elif word in universal_symbols:
            temp_comments.append((word, "univ"))

        elif word_length(word) < 3:
            temp_comments.append((word, "undef"))

        elif is_mixed(word):
            temp_comments.append((word, "undef"))

        elif any(c.isdigit() for c in word) and any(c in universal_symbols for c in word):
            temp_comments.append((word, "univ"))

        else:
          if is_mixed_word_language_model(word, eng_word_set, malayalam_suffixes) and word_length(word)>3:
              temp_comments.append((word, "mix"))


          else:
              temp_comments.append((word, "mal"))

    return temp_comments


In [ ]:
df['annotated'] = df['text'].apply(tag_comment)
df.to_csv('annotated_comments.csv', index=False)
print(df.head())

      video_id                                               text  \
0  63i4ZQcvpcQ          nayanthara kettiyathin shesham kanunnavar   
1  63i4ZQcvpcQ  00:12 which is that song playing in the backgr...   
2  63i4ZQcvpcQ                      2024 july 21n kaanunna njan🥲🥲   
3  63i4ZQcvpcQ                           15:57 what she is saying   
4  63i4ZQcvpcQ  😊 ara e script oke ezhuthiye..nice dialogues a...   

                                           annotated  
0  [(nayanthara, mal), (kettiyathin, mal), (shesh...  
1  [(00:12, univ), (which, eng), (is, eng), (that...  
2  [(2024, univ), (july, eng), (21n, undef), (kaa...  
3  [(15:57, univ), (what, eng), (she, eng), (is, ...  
4  [(😊, univ), (ara, mal), (e, undef), (script, e...  
